# Navier Stokes Equatoin

In [1]:
#!/usr/bin/env python3
"""
CutFEM Turek benchmark (DFG 2D-2) implemented with NGSolve/ngsxfem.

The cylinder is described implicitly with a level-set function on a
structured background mesh.  No-slip on the immersed boundary is
enforced by a symmetric Nitsche formulation while the inflow and wall
conditions are imposed strongly on the degrees of freedom exposed on
the outer boundary.  The temporal discretisation solves the fully
implicit backward-Euler Navier–Stokes
step with Newton's method at every time level.

The script records drag, lift and the pressure drop between the
benchmark probe points in CSV files inside ``--output-dir``.
"""

from __future__ import annotations

import argparse
import csv
from dataclasses import dataclass
from math import ceil, sqrt
from pathlib import Path

from netgen.geom2d import SplineGeometry
from ngsolve import (
    BilinearForm,
    CoefficientFunction,
    GridFunction,
    Grad,
    H1,
    Id,
    InnerProduct,
    Integrate,
    Mesh,
    Norm,
    NumberSpace,
    SetHeapSize,
    TaskManager,
    VectorH1,
    VTKOutput,
    div,
    specialcf,
    x,
    y,
    IfPos,
    FESpace,
)
from xfem import (
    HASNEG,
    CutInfo,
    POS,
    NEG,
    IF,
    HASPOS,
    dCut,
    GetFacetsWithNeighborTypes,
    SymbolicFacetPatchBFI,
)
from xfem.lsetcurv import LevelSetMeshAdaptation
from ngsolve.solvers import Newton

try:
    import matplotlib.pyplot as plt
except ImportError:  # pragma: no cover - optional dependency
    plt = None


SetHeapSize(200 * 10**6)
@dataclass
class SimulationParameters:
    rho: float
    mu: float
    dt: float
    gamma_n: float
    gamma_gp: float
    u_mean: float
    diameter: float
    n_steps: int
    cx: float 
    cy: float
    length: float
    height: float
    max_h: float
    p_reg: float = 1.0e-8
    order: int = 2
    output_dir: Path = Path("turek_benchmark_results_ngsolve")
    newton_max_it: int = 20
    newton_tol: float = 1e-8
    newton_damp: float = 1.0
    quiet_newton: bool = False
    vtk_output_interval: int = 10

importing ngsxfem-2.1.2501.dev0


In [2]:
def parabolic_inflow(height: float, mean_velocity: float) -> CoefficientFunction:
    """Return the DFG parabolic inflow profile."""
    return CoefficientFunction((4.0 * mean_velocity * y * (height - y) / (height ** 2), 0.0))


def build_background_mesh(length: float, height: float, maxh: float) -> Mesh:
    """Generate a tensor-product background mesh without the cylinder cut-out."""
    geo = SplineGeometry()
    p0 = geo.AddPoint(0.0, 0.0)
    p1 = geo.AddPoint(length, 0.0)
    p2 = geo.AddPoint(length, height)
    p3 = geo.AddPoint(0.0, height)
    geo.Append(["line", p0, p1], leftdomain=1, rightdomain=0, bc="bottom")
    geo.Append(["line", p1, p2], leftdomain=1, rightdomain=0, bc="outlet")
    geo.Append(["line", p2, p3], leftdomain=1, rightdomain=0, bc="top")
    geo.Append(["line", p3, p0], leftdomain=1, rightdomain=0, bc="inlet")
    mesh = Mesh(geo.GenerateMesh(maxh=maxh, quad_dominated=False))
    mesh.Curve(3)
    return mesh


def levelset_for_cylinder(cx: float, cy: float, radius: float) -> CoefficientFunction:
    """Signed distance to the circular obstacle (positive outside)."""
    return Norm(CoefficientFunction((x - cx, y - cy))) - radius


def symgrad(v):
    """Return the symmetric gradient tensor."""
    grad_v = Grad(v)
    return 0.5 * (grad_v + grad_v.trans)


def divergence(v):
    grad_v = Grad(v)
    dim = grad_v.dims[0]
    return sum(grad_v[i, i] for i in range(dim))


def setup_cut_geometry(mesh: Mesh, levelset_cf: CoefficientFunction, order: int):
    """Create the isoparametric deformation and CutInfo for the level-set."""
    lset_adapter = LevelSetMeshAdaptation(
        mesh,
        order=max(2, order),
        threshold=10.5,
        discontinuous_qn=True,
    )
    deformation = lset_adapter.CalcDeformation(levelset_cf)
    mesh.deformation = deformation
    lset_p1 = lset_adapter.lset_p1
    cut_info = CutInfo(mesh, lset_p1)
    return lset_p1, deformation, cut_info


def build_cut_spaces(mesh: Mesh, _cut_info: CutInfo, order: int):
    """Taylor-Hood spaces on the background mesh with jump couplings enabled."""
    V = VectorH1(mesh, order=order, dirichlet="inlet|top|bottom", dgjumps=True)
    Q = H1(mesh, order=max(1, order - 1), dgjumps=True)
    # lagrange = NumberSpace(mesh)
    # return FESpace([V, Q, lagrange], dgjumps=True)
    return FESpace([V, Q], dgjumps=True)


def build_measures(mesh: Mesh, cut_info: CutInfo, lset_p1, deformation):
    """Return volume, interface and ghost facet sets."""
    els_outer = cut_info.GetElementsOfType(HASPOS)
    els_inner = cut_info.GetElementsOfType(HASNEG)
    dx_fluid = dCut(levelset=lset_p1, domain_type=POS, 
                    deformation=deformation,
                    definedonelements=els_outer)
    dx_inside = dCut(levelset=lset_p1, domain_type=NEG, deformation=deformation, definedonelements=els_inner)
    ds_interface = dCut(levelset=lset_p1, domain_type=IF, deformation=deformation)

    positive_elements = cut_info.GetElementsOfType(HASPOS)
    interface_elements = cut_info.GetElementsOfType(IF)
    ghost_facets = GetFacetsWithNeighborTypes(mesh, a=positive_elements, b=interface_elements)
    return dx_fluid, ds_interface, ghost_facets, dx_inside
def compute_forces(
    mesh: Mesh,
    velocity: GridFunction,
    pressure: GridFunction,
    params: SimulationParameters,
    ds_interface,
    normal_cf,
):
    """Return drag, lift and their dimensionless coefficients."""
    vel_cf = CoefficientFunction(velocity)
    press_cf = CoefficientFunction(pressure)

    traction = (2.0 * params.mu * symgrad(vel_cf) - press_cf * Id(mesh.dim)) * normal_cf
    ex = CoefficientFunction((1.0, 0.0))
    ey = CoefficientFunction((0.0, 1.0))
    drag = Integrate(InnerProduct(traction, ex) * ds_interface, mesh=mesh)
    lift = -Integrate(InnerProduct(traction, ey) * ds_interface, mesh=mesh)
    coeff = 2.0 / (params.rho * (params.u_mean ** 2) * params.diameter)
    return float(drag), float(lift), float(coeff * drag), float(coeff * lift)


def compute_pressure_drop(mesh: Mesh, pressure: GridFunction, point_a, point_b) -> float:
    """Evaluate Δp between two probe points."""
    pa = pressure(mesh(*point_a, 0.0))
    pb = pressure(mesh(*point_b, 0.0))
    return float(pa - pb)


In [3]:
def build_nonlinear_form(
    space,
    params: SimulationParameters,
    measures,
    lset_p1,
    previous_velocity: GridFunction,
):
    """Create the semilinear form defining the Navier–Stokes residual."""
    dx_fluid, ds_interface, ghost_facets, dx_inside = measures
    # (u, p, lam), (v, q, eta) = space.TnT()
    lam=eta=None
    (u, p), (v, q) = space.TnT()

    vel_prev_cf = CoefficientFunction(previous_velocity)
    normal_vec = Grad(lset_p1)
    normal_vec = normal_vec / Norm(normal_vec)
    h = specialcf.mesh_size

    form = BilinearForm(space, check_unused=False)
    form += (params.rho / params.dt * InnerProduct(u - vel_prev_cf, v) * dx_fluid
            -  p * div(v) * dx_fluid
            +   q * divergence(u) * dx_fluid
            + 0.5 * 2.0 * params.mu * InnerProduct(symgrad(u), symgrad(v)) * dx_fluid
            + 0.5 * 2.0 * params.mu * InnerProduct(symgrad(vel_prev_cf), symgrad(v)) * dx_fluid
            + 0.5* params.rho * InnerProduct(Grad(u) * u, v) * dx_fluid
            + 0.5* params.rho * InnerProduct(Grad(vel_prev_cf) * vel_prev_cf, v) * dx_fluid
    )
    # form += params.p_reg * p * q * dx_fluid
    if lam is not None and eta is not None:
        form += lam * q * dx_fluid
        form += eta * p * dx_fluid

    form += (
        - 0.5 * params.mu * InnerProduct(symgrad(u) * normal_vec, v)
        - 0.5 * params.mu * InnerProduct(symgrad(v) * normal_vec, u)
        + params.gamma_n * params.mu / h * InnerProduct(u, v)
    ) * ds_interface

    if params.gamma_gp > 0.0:
        ghost_integrand = params.gamma_gp * params.mu / h ** 2 * InnerProduct(
            u - u.Other(), v - v.Other()
        )
        ghost_integrand += - params.gamma_gp / params.mu * (p - p.Other()) * (
            q - q.Other()
        )
        form += SymbolicFacetPatchBFI(
            form=ghost_integrand,
            skeleton=False,
            definedonelements=ghost_facets,
        )

    return form


In [4]:
from ngsolve.webgui import Draw
from xfem import DrawDC



params = SimulationParameters(
    rho=1.0,
    mu=0.001,
    dt=0.01,
    n_steps=40,
    gamma_n=20.0,
    gamma_gp=1e-3,
    u_mean=1.0,
    diameter=0.1,
    cx=0.2,
    cy=0.2,
    length=2.2,
    height=0.41,
    max_h=0.08,
)
mesh = build_background_mesh(params.length, params.height, params.max_h)
levelset_cf = levelset_for_cylinder(params.cx, params.cy, params.diameter * 0.5)
Draw(mesh)
DrawDC(levelset_cf, -1.0, 2.0, mesh,"x")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [5]:
lset_p1, deformation, cut_info = setup_cut_geometry(mesh, levelset_cf, params.order)
space = build_cut_spaces(mesh, cut_info, params.order)
measures = build_measures(mesh, cut_info, lset_p1, deformation)
DrawDC(lset_p1, -1.0, 2.0, mesh,"x")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [6]:
gfu = GridFunction(space, name="state")
# velocity, pressure, lagrange = gfu.components
velocity, pressure = gfu.components; lagrange=None
initial_profile = parabolic_inflow(params.height, params.u_mean)
velocity.Set(initial_profile, definedon=mesh.Boundaries("inlet"))
pressure.Set(0.0)
if lagrange is not None:
    lagrange.Set(0.0)

inflow_profile = parabolic_inflow(params.height, params.u_mean)

previous_velocity = GridFunction(space.components[0])
previous_velocity.Set(velocity, definedon=mesh.Boundaries("inlet"))
nonlinear_form = build_nonlinear_form(space, params, measures, lset_p1, previous_velocity)
freedofs = space.FreeDofs()


In [7]:
Draw(velocity, mesh=mesh, deformation=deformation)
Draw(pressure, mesh=mesh, deformation=deformation)
def plot(times, cd_hist, cl_hist, drag_hist, lift_hist, dp_hist):
        # Create a 3x2 grid of subplots.
    # `sharex=True` links the x-axes of all plots.
    # `figsize` is adjusted for a wider layout.
    fig, axes = plt.subplots(3, 2, figsize=(12, 9), sharex=True)
    
    # --- Row 0: Coefficients ---
    # Subplot (0, 0) for Drag Coefficient
    axes[0, 0].plot(times, cd_hist, label="Cd", color="blue")
    axes[0, 0].set_ylabel("Drag Coefficient (Cd)")
    axes[0, 0].grid(True, linestyle=":", linewidth=0.5)
    axes[0, 0].set_title("Drag Coefficient over Time")

    # Subplot (0, 1) for Lift Coefficient
    axes[0, 1].plot(times, cl_hist, label="Cl", color="green")
    axes[0, 1].set_ylabel("Lift Coefficient (Cl)")
    axes[0, 1].grid(True, linestyle=":", linewidth=0.5)
    axes[0, 1].set_title("Lift Coefficient over Time")

    # --- Row 1: Forces ---
    # Subplot (1, 0) for Drag Force
    axes[1, 0].plot(times, drag_hist, label="Drag", color="red")
    axes[1, 0].set_ylabel("Drag Force")
    axes[1, 0].grid(True, linestyle=":", linewidth=0.5)
    axes[1, 0].set_title("Drag Force over Time")

    # Subplot (1, 1) for Lift Force
    axes[1, 1].plot(times, lift_hist, label="Lift", color="purple")
    axes[1, 1].set_ylabel("Lift Force")
    axes[1, 1].grid(True, linestyle=":", linewidth=0.5)
    axes[1, 1].set_title("Lift Force over Time")

    # --- Row 2: Pressure Drop ---
    # Subplot (2, 0) for Pressure Drop
    axes[2, 0].plot(times, dp_hist, label="Δp", color="orange")
    axes[2, 0].set_xlabel("Time")
    axes[2, 0].set_ylabel("Pressure Drop (Δp)")
    axes[2, 0].grid(True, linestyle=":", linewidth=0.5)
    axes[2, 0].set_title("Pressure Drop over Time")

    # Subplot (2, 1) is unused, so we turn it off
    fig.delaxes(axes[2, 1])

    # Add a main title to the entire figure
    fig.suptitle("Flow Diagnostics for Turek Benchmark", fontsize=16)

    # Adjust layout to prevent titles and labels from overlapping
    fig.tight_layout(rect=[0, 0, 1, 0.96]) # rect leaves space for suptitle
    plt.show()
    # plt.close(fig)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [8]:
import time
output_dir = params.output_dir
output_dir.mkdir(parents=True, exist_ok=True)
forces_path = output_dir / "forces.csv"
pressure_path = output_dir / "pressure_drop.csv"

velocity_cf = CoefficientFunction(velocity)
velocity_mag = Norm(velocity_cf)

vtk_output = VTKOutput(
    mesh,
    coefs=[velocity, pressure, velocity_mag],
    names=["velocity", "pressure", "velocity_magnitude"],
    filename=str((output_dir / "turek_ngsolve").resolve()),
    subdivision=2,
)

normal_cf = Grad(lset_p1) / Norm(Grad(lset_p1))

times: list[float] = []
drag_hist: list[float] = []
lift_hist: list[float] = []
cd_hist: list[float] = []
cl_hist: list[float] = []
dp_hist: list[float] = []
velocity.Set(inflow_profile, definedon=mesh.Boundaries("inlet"))
velocity.Set((0.0, 0.0), definedon=mesh.Boundaries("top|bottom"))
sceneu = DrawDC(velocity, -1.0, 2.0, mesh,"x")
scenep = DrawDC(pressure, -1.0, 2.0, mesh,"x")
# sceneu = Draw(velocity, mesh=mesh)
# scenep = Draw(pressure, mesh=mesh)

with forces_path.open("w", newline="") as f_forces, pressure_path.open("w", newline="") as f_press:
    forces_writer = csv.writer(f_forces)
    pressure_writer = csv.writer(f_press)
    forces_writer.writerow(["step", "time", "drag", "lift", "cd", "cl"])
    pressure_writer.writerow(["step", "time", "delta_p"])

    current_time = 0.0
    print("--- Starting CutFEM Turek benchmark (NGSolve/ngsxfem) ---")
    for step in range(1, params.n_steps + 1):
        current_time += params.dt
        print(f"\nStep {step:03d}/{params.n_steps:03d} | t = {current_time:.3f}")

        velocity.Set(inflow_profile, definedon=mesh.Boundaries("inlet"))
        velocity.Set((0.0, 0.0), definedon=mesh.Boundaries("top|bottom"))
        # lagrange.Set(0.0)

        status, iters = Newton(
            nonlinear_form,
            gfu,
            freedofs=freedofs,
            maxit=params.newton_max_it,
            maxerr=params.newton_tol,
            dampfactor=params.newton_damp,
            printing=not params.quiet_newton,
        )
        if status != 0:
            print("  Warning: Newton did not converge within the allotted iterations.")
        elif not params.quiet_newton:
            print(f"  Newton converged in {iters} iterations.")

        drag, lift, cd, cl = compute_forces(
            mesh,
            velocity,
            pressure,
            params,
            measures[1],
            normal_cf,
        )
        delta_p = compute_pressure_drop(mesh, pressure, (0.15, 0.2), (0.25, 0.2))

        print(f"  Drag={drag:.4e}, Lift={lift:.4e}, Cd={cd:.4f}, Cl={cl:.4f}, Δp={delta_p:.4e}")
        forces_writer.writerow([step, current_time, drag, lift, cd, cl])
        pressure_writer.writerow([step, current_time, delta_p])
        f_forces.flush()
        f_press.flush()

        times.append(current_time)
        drag_hist.append(drag)
        lift_hist.append(lift)
        cd_hist.append(cd)
        cl_hist.append(cl)
        dp_hist.append(delta_p)

        vtk_output.Do(time=current_time)
        sceneu.Redraw()
        scenep.Redraw()

        previous_velocity.Set(velocity)
        time.sleep(1)
        

print("\nSimulation finished.")
plot(times, cd_hist, cl_hist, drag_hist, lift_hist, dp_hist)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

--- Starting CutFEM Turek benchmark (NGSolve/ngsxfem) ---

Step 001/040 | t = 0.010
Newton iteration  0
err =  0.25187791695375605
Newton iteration  1
err =  0.0017258690456298206
Newton iteration  2
err =  1.9380507462200958e-07
Newton iteration  3
err =  1.9140149429483964e-15
  Newton converged in 4 iterations.
  Drag=4.1040e-04, Lift=-7.4579e-04, Cd=0.0082, Cl=-0.0149, Δp=1.3888e-03

Step 002/040 | t = 0.020
Newton iteration  0
err =  0.22314690519612138
Newton iteration  1
err =  0.0014073554409185988
Newton iteration  2
err =  1.3207129313476667e-07
Newton iteration  3
err =  8.977372855693544e-16
  Newton converged in 4 iterations.
  Drag=7.7390e-04, Lift=-1.6543e-04, Cd=0.0155, Cl=-0.0033, Δp=2.2840e-02

Step 003/040 | t = 0.030
Newton iteration  0
err =  0.1997838275576662
Newton iteration  1
err =  0.0011661190241171874
Newton iteration  2
err =  8.543043789725728e-08
Newton iteration  3
err =  4.368475750369794e-16
  Newton converged in 4 iterations.
  Drag=-6.7465e-04, Lift

KeyboardInterrupt: 